## Work in progress :)

Hello, kagglers! :D Nice to meet you. In this notebook, I am going to share some cool material that I have been learning and collecting from a many different places!

# Imports

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier

# Parameters

In [ ]:
LOCAL_FLAG = False

INPUT_PATH = '/kaggle/input/tabular-playground-series-mar-2021/'
TRAIN_FILE = 'train.csv'
TEST_FILE = 'test.csv'
SUBMISSION_FILE = 'sample_submission.csv'


EARLY_STOPPING_ROUNDS = 200
NUM_FOLDS = 10
SEED = 42

In [ ]:
train = pd.read_csv(INPUT_PATH + TRAIN_FILE, index_col='id')
test = pd.read_csv(INPUT_PATH + TEST_FILE, index_col='id')
submission = pd.read_csv(INPUT_PATH + SUBMISSION_FILE, index_col='id')

In [ ]:
train.head()

# Initial Exploration

In [ ]:
train.describe()

In [ ]:
print(train.shape)
print(train.info())

In [ ]:
print(train.nunique())

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
print(test.shape)
print(test.info())

In [ ]:
print(test.nunique())

In [ ]:
submission.head()

# Label Encoding

In [ ]:
for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
train.head()

In [ ]:
target = train.pop('target')

In [ ]:
target = target.to_numpy()

In [ ]:
target.shape

In [ ]:
columns = test.columns

In [ ]:
print(columns)

In [ ]:
train = pd.read_csv(INPUT_PATH + TRAIN_FILE, index_col='id')
test = pd.read_csv(INPUT_PATH + TEST_FILE, index_col='id')

for c in train.columns:
    if train[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values) + list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)

        
target = train.pop('target')
target = target.to_numpy()

In [ ]:
def cross_valid(model, train, target, num_folds=10, random_state=42):#, init_model=pretrain_lgbm):

    train_oof = np.zeros((len(train)))
    test_preds = 0

    kf = StratifiedKFold(n_splits=num_folds, random_state=SEED, shuffle=True)
    aucs=[]

    for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):

        train_df, val_df = train.iloc[train_ind][columns], train.iloc[val_ind][columns]
        
        train_target, val_target = target[train_ind], target[val_ind]

        model.fit(train_df, 
        train_target,
        eval_set= [(train_df, train_target), (val_df, val_target)], 
        eval_metric='auc', 
        verbose=0,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        #init_model=pretrain_lgbm,
        )
        
        
        temp_oof = model.predict_proba(val_df)[:, 1]
        temp_test = model.predict_proba(test[columns])[:, 1]

        train_oof[val_ind] = temp_oof
        test_preds += temp_test/num_folds
        
        aucs.append(roc_auc_score(val_target, temp_oof))
        
        print(f'Fold {f}: {roc_auc_score(val_target, temp_oof)}')
        
    print("Mean ROC AUC Score: ", np.mean(aucs))
    
    return train_oof, test_preds, np.mean(aucs)

In [ ]:
lgb_params={
    'learning_rate': 0.01,
    'metric': 'auc',
    'n_estimators': 11000,
    'num_leaves': 20067,
    'max_depth': 27,
    'reg_alpha': 9.630576598001266,
    'reg_lambda': 2.346945113164939,
    'colsample_bytree': 0.29858836720777177,
    'subsample': 0.6267448547447422,
    'min_child_samples': 61,
    'subsample_freq': 2,
    'subsample': 0.8329687190743886,
    'max_bin': 899,
    'min_data_per_group': 73,
    'random_state': SEED,
    'n_jobs': -1,
    'bagging_seed': SEED,
    'feature_fraction_seed': SEED
}


clf1 = LGBMClassifier(**lgb_params)

train_oof_1, test_preds_1, score_oof_1 = cross_valid(clf1, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_1:0.5f}') 

In [ ]:
# np.save('train_oof_lgbm_1', train_oof_1)
# np.save('test_preds_lgbm_1', test_preds_1)

In [ ]:
clf2  = LGBMClassifier(
        n_estimators=12000,
        num_leaves=105,
        colsample_bytree=.8,
        subsample=.8,
        max_depth=7,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01
    )

train_oof_2, test_preds_2, score_oof_2 = cross_valid(clf2, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_2:0.5f}') 

In [ ]:
clf3 = LGBMClassifier(
        n_estimators=12000,
        num_leaves=50,
        colsample_bytree=.8,
        subsample=.8,
        max_depth=8,
        reg_alpha=.1,
        reg_lambda=.1,
        min_split_gain=.01
    )

train_oof_3, test_preds_3, score_oof_3 = cross_valid(clf3, train, target, num_folds=NUM_FOLDS, random_state=SEED)

In [ ]:
print(f'Score oof: {score_oof_3:0.5f}') 

In [ ]:
# clf1 = LGBMClassifier(
#         n_estimators=10000,
#         num_leaves=20,
#         colsample_bytree=.8,
#         subsample=.8,
#         max_depth=8,
#         reg_alpha=.1,
#         reg_lambda=.1,
#         min_split_gain=.01
#     )

# clf1.fit(X_train, 
#         y_train,
#         eval_set= [(X_train, y_train), (X_test, y_test)], 
#         eval_metric='auc', 
#         verbose=0, 
#         early_stopping_rounds=50
#        )

# y_pred1 = clf1.predict_proba(X_test)[:, 1]
# score1 = roc_auc_score(y_test, y_pred1)
# print(f'{score1:0.5f}') 

In [ ]:
# score1 = roc_auc_score(y_test, y_pred1)
# print(f'{score1:0.5f}') 

In [ ]:
# clf2 = LGBMClassifier(
#         n_estimators=20000,
#         num_leaves=25,
#         colsample_bytree=.8,
#         subsample=.8,
#         max_depth=7,
#         reg_alpha=.1,
#         reg_lambda=.1,
#         min_split_gain=.01
#     )#RandomForestClassifier(n_estimators=2000, max_depth=8, n_jobs=-1)

# clf2.fit(X_train, 
#         y_train,
#         eval_set= [(X_train, y_train), (X_test, y_test)], 
#         eval_metric='auc', 
#         verbose=0, 
#         early_stopping_rounds=40
#        )


# y_pred2 = clf2.predict_proba(X_test)[:, 1]

In [ ]:
# score2 = roc_auc_score(y_test, y_pred2)
# print(f'{score2:0.5f}') 

In [ ]:
# clf3 = LGBMClassifier(
#         n_estimators=20000,
#         num_leaves=30,
#         colsample_bytree=.8,
#         subsample=.8,
#         max_depth=8,
#         reg_alpha=.1,
#         reg_lambda=.1,
#         min_split_gain=.01
#     )#RandomForestClassifier(n_estimators=3000, max_depth=12, n_jobs=-1)
# clf3.fit(X_train, 
#         y_train,
#         eval_set= [(X_train, y_train), (X_test, y_test)], 
#         eval_metric='auc', 
#         verbose=0, 
#         early_stopping_rounds=30
#        )

# y_pred3 = clf3.predict_proba(X_test)[:, 1]

In [ ]:
# score3 = roc_auc_score(y_test, y_pred3)
# print(f'{score3:0.5f}')

## Simple Ensemble

In [ ]:
# from scipy.optimize import minimize

# ### finding the optimum weights

# predictions = []
# predictions.append(test_preds_1)
# predictions.append(test_preds_2)
# predictions.append(test_preds_3)

# def roc_auc_loss_func(weights):
#     ''' scipy minimize will pass the weights as a numpy array '''
#     final_prediction = 0
#     for weight, prediction in zip(weights, predictions):
#             final_prediction += weight*prediction

#     return roc_auc_score(y_test, final_prediction)

# # starting random value for each weight
# starting_values = [0.5]*len(predictions) 

# cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
# # our weights are bound between 0 and 1
# bounds = [(0,1)]*len(predictions) 

# res = minimize(roc_auc_loss_func, starting_values, method = 'Nelder-Mead', bounds=bounds, constraints=cons)

# print('Ensamble Score: {best_score}'.format(best_score=res['fun']))
# print('Best Weights: {weights}'.format(weights=res['x']))

In [ ]:
# tt1 = clf1.predict_proba(test)[:, 1]
# tt2 = clf2.predict_proba(test)[:, 1]
# tt3 = clf3.predict_proba(test)[:, 1]

In [ ]:
# submission['target'] =  (res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3 #(test_preds_1 + test_preds_2 + test_preds_3)/3 #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
# submission.to_csv('submission_nelder_mead.csv')
# submission.head()

In [ ]:
# from scipy.optimize import minimize

# ### finding the optimum weights

# predictions = []
# predictions.append(test_preds_1)
# predictions.append(test_preds_2)
# predictions.append(test_preds_3)

# def roc_auc_loss_func(weights):
#     ''' scipy minimize will pass the weights as a numpy array '''
#     final_prediction = 0
#     for weight, prediction in zip(weights, predictions):
#             final_prediction += weight*prediction

#     return roc_auc_score(y_test, final_prediction)

# # starting random value for each weight
# starting_values = [0.5]*len(predictions) 

# cons = ({'type':'eq','fun':lambda w: 1-sum(w)})
# # our weights are bound between 0 and 1
# bounds = [(0,1)]*len(predictions) 

# res = minimize(roc_auc_loss_func, starting_values, bounds=bounds, constraints=cons)

# print('Ensamble Score: {best_score}'.format(best_score=res['fun']))
# print('Best Weights: {weights}'.format(weights=res['x']))

In [ ]:
tt1 = clf1.predict_proba(test)[:, 1]
tt2 = clf2.predict_proba(test)[:, 1]
tt3 = clf3.predict_proba(test)[:, 1]

In [ ]:
# submission['target'] =  (0.33*test_preds_1 + 0.33*test_preds_2 + 0.34*test_preds_3)/3 #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
# submission.to_csv('submission.csv')
# submission.head()

In [ ]:
submission['target'] = (test_preds_1 + test_preds_2 + test_preds_3)/3 #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission.csv')
submission.head()

In [ ]:
submission['target'] =  (test_preds_1)#(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_1.csv')
submission.head()

In [ ]:
submission['target'] =  (test_preds_2) #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_2.csv')
submission.head()

In [ ]:
submission['target'] =  (test_preds_3) #(res['x'][0]*tt1+res['x'][1]*tt2+res['x'][2]*tt3)/3
submission.to_csv('submission_model_3.csv')
submission.head()